# Adding, Removing, and Changing Boundary Conditions

In [2]:
import openpnm as op
import numpy as np

Start by creating a basic 'demo' network and phase object:

In [18]:
pn = op.network.Demo(shape=[4, 4, 1])
water = op.phase.Water(network=pn)
water.add_model_collection(op.models.collections.physics.standard)
water.regenerate_models()

## Initialize a Transport Algorithm
Now initialize a transport algorithm, in this case diffusion:

In [34]:
fd = op.algorithms.FickianDiffusion(network=pn, phase=water)

All transport algorithms have two arrays pre-defined on them for storing **rate** and **value** boundary conditions, ``'pore.bc.rate'`` and ``'pore.bc.value'``. They both initially contain all ``nans`` which is interpreted to mean no boundary conditions are applied in the corresponding pores. 

In [35]:
print(fd['pore.bc.value'])

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]


In [36]:
print(fd['pore.bc.rate'])

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]


**Pro Tip**: Omitting the trailing part of a dictionary key will result in the return of a sub-dictionary containing any arrays who have the requested prefix. This can be used to fetch a dictionary containing all available boundary conditions using ``fd['pore.bc']`` as follows:

In [37]:
print(fd['pore.bc'].keys())

dict_keys(['rate', 'value'])


## Setting Value Boundary Conditions

We can apply 'value' or 'Dirchlet' boundary conditions to the pores on the 'left' side of the domain, which is conveniently already labelled by the network generator.  We can assign a value of 1.0 to *all* the pores by passing in a scalar:

In [38]:
fd.set_value_BC(pores=pn.pores('left'), values=1.0, mode='add')

Now when we inspect the 'pore.bc.value' array we can see the 1.0 added to the specified locations:

In [39]:
print(fd['pore.bc.value'])

[ 1.  1.  1.  1. nan nan nan nan nan nan nan nan nan nan nan nan]


We can also specify a unique value for each pore by passing in an array:

In [40]:
vals = np.random.rand(pn.num_pores('front'))*10.0
fd.set_value_BC(pores=pn.pores('front'), values=vals, mode='add')

Printing the 'pore.bc.value' array again reveals a problem

In [41]:
print(fd['pore.bc.value'])

[1.         1.         1.         1.         7.01518322        nan
        nan        nan 3.57992658        nan        nan        nan
 9.35366259        nan        nan        nan]


Although we added values to 4 locations, only 3 were written, and the pre-existing values of 1.0 all remained. This is because we used ``mode='add'`` for both cases, which does *not* overwrite any pre-existing values. We need to remove the values from the locations first:

In [42]:
fd.set_value_BC(pores=pn.pores('front'), mode='remove')
fd.set_value_BC(pores=pn.pores('front'), values=vals, mode='add')
print(fd['pore.bc.value'])

[6.03773194 1.         1.         1.         7.01518322        nan
        nan        nan 3.57992658        nan        nan        nan
 9.35366259        nan        nan        nan]


Now we can see that the 1.0 in pore 0 has been filled with a random number.  There is a convenient short-cut for this using ``mode='overwrite'``:

In [43]:
fd.set_value_BC(pores=pn.pores('front'), values=2.0, mode='overwrite')
print(fd['pore.bc.value'])

[ 2.  1.  1.  1.  2. nan nan nan  2. nan nan nan  2. nan nan nan]


**Pro Tip**: The mode argument can be a list of modes, which are each applied in order, so the 'overwrite' behavior can be achieved with ``mode=['remove', 'add']``.  Another useful option may be to clear all existing values the adding new ones, as follows:

In [44]:
fd.set_value_BC(pores=pn.pores('front'), values=2.0, mode=['clear', 'add'])
print(fd['pore.bc.value'])

[ 2. nan nan nan  2. nan nan nan  2. nan nan nan  2. nan nan nan]


## Adding Rate Boundary Conditions

Adding rate boundary conditions is done in exactly the same manner as value conditions described above, but via the ``set_rate_BC`` method. One special consideration when using rate boundary conditions is that they *must* be combined with a value boundary condition. Therefore, in this section we'll look at the situation of defining two different types of boundary conditions at the same time. 

When applying rate conditions to pores without any pre-existing value condtions, there is no problem. However, if one wishes to overwrite value BCs with rate BCs (or vice-versa) special treatment is required. To add rate BCs to pores that already have rate BCs one just needs to use ``mode='overwrite'``.  Howver, to add rate BCs to pores with *value* BCs then it is necessary to first remove the values from the target locations since value and rate conditions cannot exist in the same pores.  

Firstly, let's try to add rate BCs and note that nothing is written:

In [45]:
fd.set_rate_BC(pores=pn.pores('front'), rates=2.0, mode='overwrite')
print(fd['pore.bc.rate'])

------------------------------------------------------------
    SOURCE     : openpnm.algorithms._generic_algorithm.set_BC 
    TIME STAMP : 2022-06-30 21:38:59,714    
------------------------------------------------------------


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]


As can be seen the rate BCs were not written anywhere since value BCs were present in the given pores. If we first ``remove`` the values, then we can get the result we want:

In [46]:
fd.set_value_BC(pores=pn.pores('left'), mode='remove')
fd.set_rate_BC(pores=pn.pores('left'), rates=2.0, mode='overwrite')
print(fd['pore.bc.rate'])

[ 2.  2.  2.  2. nan nan nan nan nan nan nan nan nan nan nan nan]
